In [3]:
# prompt: install dash

!pip install dash==2.15.0 dash-core-components==2.0.0 dash-html-components==2.0.0 dash-table==5.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [24]:
from IPython import get_ipython
from IPython.display import display
# %%
# prompt: install dash

!pip install dash==2.15.0 dash-core-components==2.0.0 dash-html-components==2.0.0 dash-table==5.0.0
# %%
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output # Import Input and Output here

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
# Load data
spacex_df = pd.read_csv(url)

# Prepare dropdown options
launch_sites = spacex_df['Launch Site'].unique()
options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_sites]

# Determine min and max payload for slider default range
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

app = dash.Dash(__name__)

# Define the overall layout with all components
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    # Dropdown for selecting launch site
    dcc.Dropdown(
        id='site-dropdown',
        options=options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # Pie chart for success by site or success vs failure
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Slider for payload mass range
    html.P("Payload Mass (kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: f'{i}' for i in range(0, 10001, 2000)},
        value=[min_payload, max_payload]
    ),
    html.Br(),

    # Scatter plot for payload vs success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback for the pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Group by launch site and count success (class=1)
        filtered_df = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(
            filtered_df,
            names='Launch Site',
            title='Total Successful Launches by Site'
        )
    else:
        # Filter data for selected site, count success vs failure
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_counts = filtered_df['class'].value_counts().reset_index()
        success_counts.columns = ['class', 'count']
        success_counts['class'] = success_counts['class'].replace({0: 'Failure', 1: 'Success'})

        fig = px.pie(
            success_counts,
            names='class',
            values='count',
            title=f'Success vs Failure for site {selected_site}'
        )
    return fig


# The scatter plot callback function:
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range

    # Check if payload_range is valid
    if not isinstance(payload_range, list) or len(payload_range) != 2:
        print(f"Warning: Invalid payload_range received: {payload_range}")
        # Return an empty figure or a placeholder
        return {} # Return an empty figure

    low, high = payload_range

    if selected_site == 'ALL':
        # Filter data based on payload range only
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                                (spacex_df['Payload Mass (kg)'] <= high)]
        title = 'Payload vs Success for All Sites'
    else:
        # Filter data based on both site and payload range
        filtered_df = spacex_df[(spacex_df['Launch Site'] == selected_site) &
                                (spacex_df['Payload Mass (kg)'] >= low) &
                                (spacex_df['Payload Mass (kg)'] <= high)]
        title = f'Payload vs Success for site {selected_site}'

    # Check if filtered_df is empty
    if filtered_df.empty:
        print(f"No data found for selected site '{selected_site}' and payload range [{low}, {high}].")
        # Return an empty figure or a placeholder
        return {} # Return an empty figure


    # Create scatter plot
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=title,
        labels={'class': 'Launch Outcome (0=Failure, 1:Success)'},
        # Replace 'Flight Number' with the desired column name for hover data
        hover_data=['Flight Number']
    )

    return fig

# Run the server only once
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

<IPython.core.display.Javascript object>